#Downloading the Audio from Youtube

In [3]:
!pip install pytube -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00


In [4]:
from pytube import YouTube

In [5]:
VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo"

In [6]:
yt = YouTube(VIDEO_URL)

In [7]:
yt.streams\
          .filter(only_audio = True, file_extension= 'mp4') \
          .first() \
          .download(filename = 'ytaudio.mp4') \



'/content/ytaudio.mp4'

In [8]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

#ASR with HuggingSound

In [9]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does

In [10]:
from huggingsound import SpeechRecognitionModel

In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
device

'cpu'

##Downloading the model for English from huggingface

In [13]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


##Audio Chunking

In [14]:
import librosa

In [15]:
input_file = '/content/ytaudio.wav'

In [16]:
input_file

'/content/ytaudio.wav'

In [17]:
print(librosa.get_samplerate(input_file))

stream = librosa.stream(
    input_file,
    block_length= 30,
    frame_length=16000,
    hop_length=16000
)

16000


In [18]:
import soundfile as sf

In [19]:
for i,speech in enumerate(stream):
    sf.write(f'{i}.wav', speech, 16000)

In [20]:
audio_path = []
for a in range(i+1):
    audio_path.append(f'/content/{a}.wav')

In [21]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav']

In [22]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 7/7 [04:41<00:00, 40.26s/it]


In [23]:
full_transcript = ' '

In [24]:
for item in transcriptions:
    full_transcript += ''.join(item['transcription'])

In [25]:
full_transcript

" for even as we celebrate tonight we know the challenges that tomorrow will bring are the greatest imorlitai stand here knowing that my story is part of the larger american story ti owe a debt to all of those who came before me and that in no other country on earth is my story even possibleiam a sonablackman from kenya and white woman from kansas i am married to a black american who carries within her the blood of slaves and slave ownersi have brothers sisters nieces nephew's uncles and cousins of every race and every hugh scattered across three continents these people are part of meand they are part of america this county that i lovei've come here to cairoto seek a new beginning between the united states and muslims around the worldone based on mutual interest and mutual respect just as muslims do not fit a cruidstereotype america is not the cruidstereotype of a self-interested empireto say that force may sometimes be necessary is not a call to synicism it is a recognition of history

##Text Summarization(Ignore)


In [26]:
from transformers import pipeline

In [27]:
summarization(full_transcript)

NameError: ignored

In [ ]:
summarization = pipeline('summarization')

In [ ]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i+1)*1000
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
  summarized_text.append(out)


